In [2]:
import pandas as pd
import numpy as np
import sys
import csv
import os
params = {}
##数据集文件夹路径
params['data_path']=r'E:\文件夹\大三下\移动互联网课设\大作业\408训练\data'
###数据清洗后文件夹路径
params['iso_path']=r'E:\文件夹\大三下\移动互联网课设\大作业\408训练\isolation'
###数据归一化后文件夹路径
params['normal_path']=r'E:\文件夹\大三下\移动互联网课设\大作业\408训练\normalization'
###三种特征提取方法csv文件路径，实际要用到的是time_fre_feature（时+频域特征提取）
params['time+fre_feature_path']=r'E:\文件夹\大三下\移动互联网课设\大作业\408训练\feature\time+fre.csv'

In [4]:
###给数据集加label列并删除BA_time，RPM列
filelists = os.listdir(params['data_path'])
label=3
for info in filelists:
    ###给特征添加label
    if 'B'in info:
        label=1
    elif 'IR'in info:
        label=3
    elif 'NORMAL'in info:
        label=0
    elif 'OR'in info:
        label=2
    else:###即测试集状态
        label+=1
    params['path']= os.path.join(params['data_path'],info)
    df_data = pd.read_csv(params['path'])
    ##删除BA_time,RPM列
    df_data=df_data.drop(['BA_time','RPM'],axis=1)###删除什么列
    ###获取一列长度
    data=np.array(df_data)
    lenth=data[:,-1]
    lenth=len(np.array(lenth))
    df_data['label']=[label]*lenth
    df_data.to_csv(params['path'],index =False)

In [3]:
#########数据清洗
#####1
def data_delete(params):
    with open(params['path'],'r') as f:
    #1.创建阅读器对象
        reader = csv.reader(f)
    #2.读取文件第一行数据
        head_row=next(reader)
    data_attribute = []
    for item in head_row:
        data_attribute.append(item)

 #读取数据并删除最后一列标签
    tn = pd.read_csv(params['path']) 
    tn.dropna(inplace=True)
    train = np.array(tn)
    train_x = train[:, :-1]

#存标签
    train_y = train[:,-1]
    train_y = np.array(train_y)

#对所有数据行进行异常检测
    train_x = np.array(train_x)
    clf = IsolationForest(n_estimators=params['n_estimators'], 
                      max_samples=params['max_samples'], 
                      contamination=params['contamination'], 
                      max_features=params['max_features'], 
                      bootstrap=False, n_jobs=1, random_state=None, 
                      verbose=0).fit(train_x)
#pred存入的是每一行数据的预测值，是1或者-1
    pred = clf.predict(train_x)
    normal = train_x[pred == 1]
    abnormal = train_x[pred == -1]

#删除pred为-1的行数据
    df = pd.DataFrame(pd.read_csv(params['path']))[0:pred.size]
    df['pred']=pred

#data2=data1[-data1.sorce.isin([61])]
    df2 = df[-df.pred.isin([-1])]
    df2 = df2.drop(['pred'],axis=1)

#将清洗之后的数据存入csv文件
    data_out = df2.iloc[:,:].values
    csvfile2 = open(params['opath'],'w',newline='')
    writer = csv.writer(csvfile2)
    writer.writerow(data_attribute)   #存属性
    m = len(data_out)
    for i in range(m):
        writer.writerow(data_out[i])
        
##数据清洗
from sklearn.ensemble import IsolationForest
params['n_estimators'] = 100
params['max_samples'] ='auto'
params['contamination'] = 0.1
params['max_features'] = 1.0
# n_estimators : int, optional (default=100)森林中树的颗数
# max_samples : int or float, optional (default=”auto”)对每棵树，样本个数或比例
# contamination : float in (0., 0.5), optional (default=0.1)这是最关键的参数，用户设置样本中异常点的比例
# max_features : int or float, optional (default=1.0)对每棵树，特征个数或比例

filelists = os.listdir(params['data_path'])
for info in filelists:
    params['path']= os.path.join(params['data_path'],info)
    params['opath'] =os.path.join(params['iso_path'],info)
    data_delete(params)

In [4]:
####数据归一化
######2

def normalization(params):
    with open(params['path'],'r') as f:
    #1.创建阅读器对象
        reader = csv.reader(f)
    #2.读取文件第一行数据
        head_row=next(reader)
    data_attribute = []
    for item in head_row:
        data_attribute.append(item)

    #读取数据并删除最后一列标签
    tn = pd.read_csv(params['path']) 
    tn.dropna(inplace=True)
    train = np.array(tn)
    train_x = train[:, :-1]

    train_y = train[:,-1]
    train_y = np.array(train_y)

    #对所有数据行进行标准化
    min_max_scaler = preprocessing.MinMaxScaler(feature_range=params['feature_range'])

    X_train_minmax = min_max_scaler.fit_transform(train_x)

    out=np.column_stack((X_train_minmax,train_y))

    csvfile2 = open(params['opath'],'w',newline='')
    writer = csv.writer(csvfile2)
    writer.writerow(data_attribute)   #存属性
    m = len(out)

    for i in range(m):
         writer.writerow(out[i])

##数据归一化
from sklearn import preprocessing 
params['feature_range'] = (0,1)

filelists = os.listdir(params['iso_path'])
for info in filelists:
    params['path']= os.path.join(params['iso_path'],info)
    params['opath'] =os.path.join(params['normal_path'],info)
    normalization(params)

In [5]:
####时域+频域特征提取
######3
def time_freq_feature(params):
    df_data = pd.DataFrame(pd.read_csv(params['path']))
    sum=len(df_data)
    result_out=[]
    for r in range (0,sum,params['len_piece']):
        df=df_data[r:r+params['len_piece']]
        result_list = []
        
        arr_add=df.loc[:,:]
        for j in range(int(255/params['len_piece'])):     
            arr_add =arr_add.append(df,ignore_index=True)
        #时域
        for i in df.columns[:-1]:#除了label列的每一列
            list_para=[]
            if params['avg'] ==1:
                list_para.append(df[i].mean())
            if params['std'] ==1:
                list_para.append(df[i].std())
            if params['var'] ==1:
                list_para.append(np.var(df[i]))
            if params['skew'] ==1:
                list_para.append(stats.skew(df[i]))
            if params['kur'] ==1:
                list_para.append(stats.kurtosis(df[i]))
            if params['ptp'] ==1:
                list_para.append(df[i].ptp())
        #频域  
            flist,plist = signal.welch(arr_add[i],25600)
            list_para.append(np.square(plist[np.logical_and(flist>=params['min_fre'],flist<params['max_fre'])]).sum())
            result_list.extend(list_para)
        list_label=[df.iloc[0,-1]]
        result_list.extend(list_label) 
        result_out.append(result_list)
    return result_out

In [6]:
###时+频域特征提取
from pywt import wavedec
import math
from scipy import signal
from scipy import stats
##时域
params['avg'] = 1
params['std'] = 1
params['var'] = 1
params['skew'] = 1
params['kur'] = 1
params['ptp'] = 1
#频域    
params['min_fre']=1600
params['max_fre']=2400
###时间窗长度，应该是短精确度高，长运行快
params['len_piece']=10

filelists = os.listdir(params['normal_path'])

##加头

with open(os.path.join(params['normal_path'],filelists[0]),'r') as f:
    reader = csv.reader(f)
    head_row=next(reader)
    f.close()
    
data_attribute=[]
for item in head_row[:-1]:###label列不执行下列操作
    ###时域头
    if params['avg'] == 1:
        data_attribute.append(item+'_avg')
    if params['std'] == 1:
        data_attribute.append(item+'_std')
    if params['var'] == 1:
        data_attribute.append(item+'_var')
    if params['skew'] == 1:
        data_attribute.append(item+'_skew')
    if params['kur'] == 1:
        data_attribute.append(item+'_kur')
    if params['ptp'] == 1:
        data_attribute.append(item+'_ptp')   
    ###频域头
    data_attribute.append(item+'_main_ener')
  
# headlabel_out=[params['head_label']]
headlabel_out=[head_row[-1]]
data_attribute.extend(headlabel_out)
head_out=[]
head_out.append(data_attribute)
##写入标题
pd.DataFrame(head_out).to_csv(params['time+fre_feature_path'],index=False,header=False)

for info in filelists:
    params['path']= os.path.join(params['normal_path'],info)
    #写入数据
    pd.DataFrame(time_freq_feature(params)).to_csv(params['time+fre_feature_path'],mode='a',index=False,header=False)

C:\Users\Administrator\.conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:28: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
C:\Users\Administrator\.conda\envs\tensorflow\lib\site-packages\scipy\signal\spectral.py:1969: UserWarning: nperseg = 256 is greater than input length  = 130, using nperseg = 130
  .format(nperseg, input_length))
C:\Users\Administrator\.conda\envs\tensorflow\lib\site-packages\scipy\signal\spectral.py:1969: UserWarning: nperseg = 256 is greater than input length  = 52, using nperseg = 52
  .format(nperseg, input_length))
C:\Users\Administrator\.conda\envs\tensorflow\lib\site-packages\scipy\signal\spectral.py:1969: UserWarning: nperseg = 256 is greater than input length  = 182, using nperseg = 182
  .format(nperseg, input_length))
C:\Users\Administrator\.conda\envs\tensorflow\lib\site-packages\scipy\signal\spectral.py:1969: UserWarning: nperseg = 256 is greater than input length  = 78, using nper